# Investigate the icustay table in the database (mimic-iv)


In [1]:
import getpass
import json
import math
import os
import psycopg2
import pandas as pd
import time

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from configobj import ConfigObj
from multiprocessing import Pool, RLock
from tqdm import tqdm
from typing import Tuple

from projects.utils import *
from projects.common import *


In [2]:
db_dir = os.path.abspath('') + "/../../../db"

(query_schema_core,
 query_schema_hosp,
 query_schema_icu,
 query_schema_derived,
 conn) = connect_to_database(db_dir)


Database: mimiciv
Username: mimiciv
>>>>> Connected to DB <<<<<


# Table columns

Table for icustays:  
['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los'] 

Table for transfers:  
['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit', 'intime', 'outtime'] 

Table for patients:  
['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']

Table for admissions:  
['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'hospital_expire_flag']

In [3]:
patients_df = get_database_table_as_dataframe(conn, query_schema_core, 'patients')
admissions_df = get_database_table_as_dataframe(conn, query_schema_core, 'admissions')
transfers_df = get_database_table_as_dataframe(conn, query_schema_core, 'transfers').sort_values(by=['intime', 'outtime'])
icustays_df = get_database_table_as_dataframe(conn, query_schema_icu, 'icustays').sort_values(by=['intime', 'outtime'])

assert len(patients_df.to_numpy()[:, 0]) == len(np.unique(patients_df.to_numpy()[:, 0])) 
assert len(admissions_df.to_numpy()[:, 1]) == len(np.unique(admissions_df.to_numpy()[:, 1])) 
assert len(icustays_df.to_numpy()[:, 2]) == len(np.unique(icustays_df.to_numpy()[:, 2])) 

patients_list = patients_df['subject_id'].tolist()
admissions_list = admissions_df['hadm_id'].tolist()

Getting patients data
Number of entries for patients : 382278
Column names : ['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']
Getting admissions data
Number of entries for admissions : 523740
Column names : ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'hospital_expire_flag']
Getting transfers data
Number of entries for transfers : 2189535
Column names : ['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit', 'intime', 'outtime']
Getting icustays data
Number of entries for icustays : 76540
Column names : ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los']


# Sample patient admission

In [4]:
transfers_df[transfers_df['hadm_id'] == admissions_list[484980]]


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
2176896,10002428,28662225.0,32007337,ED,Emergency Department,2156-04-12 09:56:00,2156-04-12 14:17:27
2037419,10002428,28662225.0,38519116,admit,Medicine,2156-04-12 14:17:27,2156-04-12 16:00:59
2037427,10002428,28662225.0,37824512,transfer,Emergency Department Observation,2156-04-12 16:00:59,2156-04-12 16:24:18
2037430,10002428,28662225.0,33987268,transfer,Medical Intensive Care Unit (MICU),2156-04-12 16:24:18,2156-04-17 15:57:08
2037587,10002428,28662225.0,32875370,transfer,Medicine,2156-04-17 15:57:08,2156-04-19 18:11:19
2037642,10002428,28662225.0,38875437,transfer,Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-19 18:13:35
2037643,10002428,28662225.0,30025354,transfer,Medical Intensive Care Unit (MICU),2156-04-19 18:13:35,2156-04-23 15:16:45
2037753,10002428,28662225.0,38277070,transfer,Medical Intensive Care Unit (MICU),2156-04-23 15:16:45,2156-04-26 18:58:41
2037856,10002428,28662225.0,31606213,transfer,Medicine,2156-04-26 18:58:41,2156-04-29 16:26:31
1937753,10002428,28662225.0,37069433,discharge,None,2156-04-29 16:26:31,NaT


In [5]:
patients_df = get_database_table_as_dataframe(conn, query_schema_core, 'patients')
admissions_df = get_database_table_as_dataframe(conn, query_schema_core, 'admissions')
transfers_df = get_database_table_as_dataframe(conn, query_schema_core, 'transfers').sort_values(by=['intime', 'outtime'])
icustays_df = get_database_table_as_dataframe(conn, query_schema_icu, 'icustays').sort_values(by=['intime', 'outtime'])

assert len(patients_df.to_numpy()[:, 0]) == len(np.unique(patients_df.to_numpy()[:, 0])) 
assert len(admissions_df.to_numpy()[:, 1]) == len(np.unique(admissions_df.to_numpy()[:, 1])) 
assert len(icustays_df.to_numpy()[:, 2]) == len(np.unique(icustays_df.to_numpy()[:, 2])) 

patients_list = patients_df['subject_id'].tolist()
admissions_list = admissions_df['hadm_id'].tolist()

Getting patients data
Number of entries for patients : 382278
Column names : ['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']
Getting admissions data
Number of entries for admissions : 523740
Column names : ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'hospital_expire_flag']
Getting transfers data
Number of entries for transfers : 2189535
Column names : ['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit', 'intime', 'outtime']
Getting icustays data
Number of entries for icustays : 76540
Column names : ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los']


In [6]:
icustays_df[icustays_df['hadm_id'] == admissions_list[484980]]


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
70655,10002428,28662225,33987268,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-12 16:24:18,2156-04-17 15:57:08,4.981134
75460,10002428,28662225,38875437,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894


# Validate the icustays with the transfers.

In [7]:
icustays_df[icustays_df['hadm_id'] == admissions_list[484980]].iloc[0]['los']

4.981134259259259

In [8]:
from tqdm import tqdm
# from tqdm.auto import tqdm  # notebook compatible


careunits = icustays_df[['first_careunit', 'last_careunit']].to_numpy()
careunits = np.unique(careunits).tolist()


err_1, err_2 = [], []

for hadm_id in tqdm(admissions_list):
    _df = transfers_df[transfers_df['hadm_id'] == hadm_id]
    _icu_df = icustays_df[icustays_df['hadm_id'] == hadm_id]

    _start, _end, _flag = [], [], 0
    for idx, cu_i in enumerate(_df['careunit']):

        if cu_i in careunits and _flag == 0:
            _start.append(_df.iloc[idx]['intime'])
            _flag += 1

        if cu_i not in careunits and _flag == 1:
            _end.append(_df.iloc[idx-1]['outtime'])
            _flag = 0

    if len(_start) > len(_end):
        _end.append(_df.iloc[-1]['outtime'])

    if len(_start) != len(_icu_df):
        err_1.append(hadm_id)
        # print(f"1. hadm entry between transfers and icustays "
        #       f"is not correct: {hadm_id}")
    else:
        for s, e, i, o in zip(_start,
                              _end,
                              _icu_df['intime'].to_list(),
                              _icu_df['outtime'].to_list()):
            if s == i and e == o:
                continue
            # print(f"2. hadm entry between transfers and icustays "
            #       f"is not correct: {hadm_id}")
            err_2.append(hadm_id)


100%|██████████| 523740/523740 [28:55<00:00, 301.76it/s]


In [12]:
# json.dump(err_1, open("unequal_merged_icu_entries_between_icustays_transfers.json", 'w+'))
# json.dump(err_2, open("unequal_intime_outtime_between_icustays_transfers.json", 'w+'))


In [29]:
icustays_df[icustays_df['hadm_id'] == err_1[1]]


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
7592,17374121,27667114,37912371,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2115-07-24 14:00:36,2115-07-24 14:50:33,0.034688


In [30]:
transfers_df[transfers_df['hadm_id'] == err_1[1]]


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
176494,17374121,27667114.0,33588312,admit,Coronary Care Unit (CCU),2115-07-22 09:50:09,2115-07-22 14:42:22
176503,17374121,27667114.0,35428124,transfer,PACU,2115-07-22 14:42:22,2115-07-23 10:32:48
176538,17374121,27667114.0,37739556,transfer,Medicine/Cardiology,2115-07-23 10:32:48,2115-07-24 14:00:36
176576,17374121,27667114.0,37912371,transfer,Coronary Care Unit (CCU),2115-07-24 14:00:36,2115-07-24 14:50:33
176578,17374121,27667114.0,32783367,transfer,Medicine/Cardiology,2115-07-24 14:50:33,2115-07-24 17:57:17
52498,17374121,27667114.0,32973608,discharge,None,2115-07-24 17:57:17,NaT
